In [12]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from scipy.stats import norm
#from scipy.stats import linregress
#import statsmodels.formula.api as smf
import csv
import scipy.optimize as opt
import pandas_datareader as pdr
from datetime import datetime


In [2]:
# function to read in data from a csv
def readPricesCSV(file):
    df = pd.read_csv(file)
    dfPrices = df["Close"]
    return dfPrices

In [213]:
# function to read in stock prices from yahoo finance
def readPricesYF(start_date, end_date, tickers):
    stockPrices = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)   # read in the data
    stockPrices = stockPrices.filter(like='Adj Close')   # tickers are columns and data values are adjusted closing prices with the date as the index
    stockPrices.columns = tickers   # change column names to be their tickers
    stockPrices = stockPrices.stack().swaplevel().sort_index().reset_index()    # make the data tall and clean it up so it is easier to perform analysis on
    stockPrices.columns = ['Firm','Date','Adj Close']     # rename the columns
#    stockPrices['Return'] = stockPrices.groupby('Firm')['Adj Close'].pct_change()    # get the daily returns for each ticker
    return stockPrices


In [253]:
# the objective function
def obj(weights):
    
    #get the return measure given by the weights and mean
    for weight,mean in weights,means:
        sum_return += weight*mean
    num_stocks = len(means)
    
    #get the risk measure given by the weights
    risk_measure = np.matmul(np.matmul(np.transpose(np.array(weights)), covMatr), np.array(weights))

    #return the objective function value
    return -(sum_return - (risk_level * risk_measure))


In [214]:
#choose which stocks to include in the optimization of the portfolio (list the tickers)
tickers = ['MSFT', 'AAPL', 'AMZN', 'NFLX']
tickers = sorted(tickers)

#set the start and end dates- choose last six months in this case
start_date = datetime(2019, 10, 30) #year, month, day
end_date = datetime(2020,10,29) #year, month, day

stockPrices = readPricesYF(start_date, end_date, tickers)    # use the function defined above to get data from Yahoo Finance


In [207]:
stockPrices

,Firm,Date,Adj Close
0,AAPL,2019-10-30,60.024124
1,AAPL,2019-10-31,61.381241
2,AAPL,2019-11-01,63.123295
3,AAPL,2019-11-04,63.537827
4,AAPL,2019-11-05,63.446533
...,...,...,...
1007,NFLX,2020-10-23,488.279999
1008,NFLX,2020-10-26,488.239990
1009,NFLX,2020-10-27,488.929993
1010,NFLX,2020-10-28,486.239990


In [237]:
# get the covariance matrix

num_stocks = len(tickers)   # the number of stocks
prices_per_stock = stock_prices_format.shape   # the number of stock prices

stock_prices_format = stockPrices.set_index(['Date', 'Firm']).unstack()   # change format so that date and each company are columns
stock_prices_format.reset_index()
#stock_prices_format.iloc[:,[0,1,2,3]]
#stock_prices_format.iloc[:,[0]]
#prices = stock_prices_format.iloc[:,[0,1,2,3]]
#prices = stockPrices.reset_index().drop(['Date','index'], axis=1).set_index('Firm').unstack()   #.drop('index', axis=1)
#prices

cov_matr = np.cov(np.array(stock_prices_format.iloc[:,range(0,num_stocks)].T))   # get the columns and transpose it so it is in right format, then turn it into covariance matrix
cov_matr

array([[   399.94976458,  10885.53495809,    468.12612002,
          1347.48843763],
       [ 10885.53495809, 339169.62274759,  14180.02727035,
         43734.04875829],
       [   468.12612002,  14180.02727035,    633.61315526,
          1870.90738611],
       [  1347.48843763,  43734.04875829,   1870.90738611,
          6020.3442951 ]])

In [262]:
# get the mean price for each stock

means = np.array(stock_prices_format.mean().to_list())    # get means for each column, convert to a list, convert to a numpy array
means

array([  85.49594642, 2444.96952199,  180.35857913,  413.54229218])

In [259]:
# get the optimization ready
risk_level = 1   # risk loving < 0; risk neutral = 0; risk averse > 0
weights = np.array([0.25,0.25,0.25,0.25])

lin_constr = opt.LinearConstraint([1,1,1,1], [1], [1])    # sum of all values are greater than or equal to 1 and less than or equal to 1, so the sum has to be equal to 1
bounds = opt.Bounds([0,0,0,0], [0.5,0.5,0.5,0.5])    # each portfolio weight is greater than 0 and less than 0.5
result = opt.minimize(obj, x0=(0.25, 0.25, 0.25, 0.25), method="trust-constr", constraints = lin_constr, bounds=bounds)
result

ValueError: too many values to unpack (expected 2)